In [1]:
from kafka import KafkaProducer
from json import dumps
from kafka import KafkaConsumer
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0"
spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

In [3]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-8_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-421af31b-e2ca-411c-b7f5-d56cabd1fe54;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6

In [4]:
KAFKA_BROKER1 ='kafka-1:9092'
KAFKA_BROKER2 ='kafka-2:9092'
KAFKA_TOPIC='tt'
kafkaMessages = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", f'{KAFKA_BROKER1},{KAFKA_BROKER2}') \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .option("includeHeaders", "true") \
  .load()

In [26]:
from pyspark.sql.functions import col,from_json

In [28]:
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([ 
    StructField("id",StringType(),True), 
    StructField("master_id",StringType(),True), 
    StructField("sku",StringType(),True), 
  ])

In [30]:
data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*").take(1)

[Row(id='1379981', master_id='1379981', sku='2411205315673')]

In [23]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Product')

In [24]:
data.printSchema()

root
 |-- value: string (nullable = true)



In [32]:
sample_data = data.select('value').take(1)

In [49]:
sample_data = sample_data[0].value

In [50]:
from json import loads

In [55]:
data_sample = loads(sample_data)

In [56]:
for key in data_sample:
    print(key,type(data_sample[key]))

id <class 'int'>
master_id <class 'int'>
sku <class 'str'>
name <class 'str'>
url_key <class 'str'>
url_path <class 'str'>
short_url <class 'str'>
type <class 'str'>
book_cover <class 'NoneType'>
short_description <class 'str'>
price <class 'int'>
list_price <class 'int'>
original_price <class 'int'>
badges <class 'list'>
badges_new <class 'list'>
discount <class 'int'>
discount_rate <class 'int'>
rating_average <class 'float'>
review_count <class 'int'>
review_text <class 'str'>
favourite_count <class 'int'>
thumbnail_url <class 'str'>
has_ebook <class 'bool'>
inventory_status <class 'str'>
inventory_type <class 'str'>
productset_group_name <class 'str'>
is_fresh <class 'bool'>
seller <class 'NoneType'>
is_flower <class 'bool'>
has_buynow <class 'bool'>
is_gift_card <class 'bool'>
salable_type <class 'NoneType'>
data_version <class 'int'>
day_ago_created <class 'int'>
all_time_quantity_sold <class 'int'>
meta_title <class 'str'>
meta_description <class 'str'>
meta_keywords <class 'str

In [8]:
# fileStream = message.writeStream \
#                       .trigger(processingTime='60 seconds')\
#                       .queryName("Persist the processed data") \
#                       .outputMode("append") \
#                       .format("parquet") \
#                       .option("path", "hdfs://namenode:9000/tiki/Product/") \
#                       .option("checkpointLocation", "hdfs://namenode:9000/tiki/checkpoints") \
#                       .start() \
#                       .awaitTermination()

In [14]:
# kafkaMessages.writeStream.show()

In [9]:
message = kafkaMessages.selectExpr("*")

In [10]:
rawQuery = message \
        .writeStream \
        .queryName("dasd")\
        .format("memory")\
        .start()

22/10/24 17:07:18 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c5165fc0-dd6a-411d-94c8-35843ff0ee04. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [16]:
spark.sql("SELECT cast(value as string) FROM dasd limit 1").collect()

[Row(value='{"id": 53789448, "master_id": 53789448}')]

In [17]:
fileStream = message.writeStream \
                      .trigger(processingTime='5 seconds')\
                      .queryName("Persist the processed data") \
                      .outputMode("append") \
                      .format("parquet") \
                      .option("path", f"hdfs://namenode:9000/tiki/tt") \
                      .option("checkpointLocation", "hdfs://namenode:9000/tiki/checkpointss") \
                      .start() \
                      .awaitTermination()

KeyboardInterrupt: 